# 图像和办公文档处理
---
用程序来处理图像和办公文档经常出现在实际开发中，Python的标准库中虽然没有直接支持这些操作的模块，但我们可以通过Python生态圈的第三方模块来完成这些操作。  

[官网手册](https://pillow.readthedocs.io/en/stable/handbook/index.html)  
[Python图像处理库简单示例](https://www.ctolib.com/topics-515.html)

Pillow中最重要的类就是Image,该类存在于同名的模块中。可以通过以下几种方式实例化。
1. 从文件中读取图片
1. 处理其他图片得到
1. 直接创建一个图片  


In [1]:
from PIL import Image

image = Image.open('./res/BingWallpaper-2019-05-21.jpg')
print(image.format, image.size, image.mode)
image.show()

JPEG (1920, 1200) RGB


![](./res/imageShow.jpg)

format属性定义了图像的格式，如果图像不是从文件打开的，那么该属性值为None；size属性是一个tuple，表示图像的宽和高（单位为像素）；mode属性为表示图像的模式，常用的模式为：L为灰度图，RGB为真彩色，CMYK为pre-press图像。  
如果文件不能打开，则抛出IOError异常。  
当有一个Image对象时，可以用Image类的各个方法进行处理和操作图像，例如显示图片：
```
im.show()
```

ps：标准版本的show()方法不是很有效率，因为它先将图像保存为一个临时文件，然后使用xv进行显示。如果没有安装xv，该函数甚至不能工作。但是该方法非常便于debug和test。（windows中应该调用默认图片查看器打开）  


## 读写图片
Pillow库支持相当多的图片格式。直接使用Image模块中的open()函数读取图片，而不必先处理图片的格式，Pillow库自动根据文件决定格式。  
Image模块中的save()函数可以保存图片，除非你指定文件格式，那么文件名中的扩展名用来指定文件格式。

### 图片转成jpg格式
关于sys.argv在[知乎](https://www.zhihu.com/question/23711222)的解释。

In [ ]:
from __future__ import print_function
import os, sys
from PIL import Image

for infile in sys.argv[1:]:
    f, e = os.path.splitext(infile)
    outfile = f + '.jpg'
    if infile != outfile:
        try:
            Image.open(infile).save(outfile)
        except IOError:
            print("cannot convert", infile)
            

### 创建缩略图

In [2]:
from PIL import Image
import os, sys

infile = './res/BingWallpaper-2019-05-21.jpg'
outfile = os.path.splitext(infile)[0] + "thumbnail.png"
size = (128, 128)
try:
    im = Image.open(infile)
    im.thumbnail(size)
    im.save(outfile, 'PNG')
except IOError:
    print('Cannot create thumbnail for', infile)

必须指出的是除非必须，Pillow不会解码或raster数据。当你打开一个文件，Pillow通过文件头确定文件格式，大小，mode等数据，余下数据直到需要时才处理。

这意味着打开文件非常快，与文件大小和压缩格式无关。下面的程序用来快速确定图片属性：

### 确定图片属性

In [ ]:
from __future__ import print_function
import sys
from PIL import Image
for infile in sys.argv[1:]:
    try:
        with Image.open(infile) as im:
            print(infile, im.format, "%dx%d" % im.size, im.mode)
    except IOError:
        pass

### 裁剪、粘贴、与合并图片
Image类包含还多操作图片区域的方法。如crop()方法可以从图片中提取一个子矩形  

从图片中复制子图像
```
box = im.copy() #直接复制图像
box = (100, 100, 400, 400)
region = im.crop(box)
```
区域由4-tuple决定，该tuple中信息为(left, upper, right, lower)。 Pillow左边系统的原点（0，0）为图片的左上角。坐标中的数字单位为像素点，所以上例中截取的图片大小为300*300像素^2。  

处理子图，粘贴回原图
```
region = region.transpose(Image.ROTATE_180)
im.paste(region, box)
```

将子图paste回原图时，子图的region必须和给定box的region吻合。该region不能超过原图。而原图和region的mode不需要匹配，Pillow会自动处理。  

另一个例子:

In [ ]:
Rolling an image
def roll(image, delta):
    "Roll an image sideways"
    image = image.copy() #复制图像
    xsize, ysize = image.size
    delta = delta % xsize
    if delta == 0: return image
    part1 = image.crop((0, 0, delta, ysize))
    part2 = image.crop((delta, 0, xsize, ysize))
    image.paste(part2, (0, 0, xsize-delta, ysize))
    image.paste(part1, (xsize-delta, 0, xsize, ysize))
    return image

### 分离和合并通道
```
r, g, b = im.split()
im = Image.merge("RGB", (b, g, r))
```
对于单通道图片，split()返回图像本身。  
为了处理单通道图片，必须先将图片转成RGB。
### 几何变换
Image类有resize()、rotate()和transpose()、transform()方法进行几何变换。
#### 简单几何变换
```
out = im.resize((128, 128))
out = im.rotate(45) # 顺时针角度表示
```
##### 置换图像
```
out = im.transpose(Image.FLIP_LEFT_RIGHT)
out = im.transpose(Image.FLIP_TOP_BOTTOM)
out = im.transpose(Image.ROTATE_90)
out = im.transpose(Image.ROTATE_180)
out = im.transpose(Image.ROTATE_270)
```
transpose()和象的rotate()没有性能差别。
更通用的图像变换方法可以使用transform().

**docx 安装：**  
打开Anaconda Promt - pip install python-docx [教程](https://blog.csdn.net/qq_41185868/article/details/80468657)  
**PyPDF2 安装：**  
打开Anaconda Promt - pip install PyPDF2 [教程](https://www.jianshu.com/p/e3d384e51db9)